In [21]:
import IPython
url = '//www.slideshare.net/slideshow/embed_code/key/dQ0Bh8mNKQ01Ht'
iframe = '<iframe src=' + url + ' width=595 height=485></iframe>'
IPython.display.HTML(iframe)

# Intro

Bsed on exploring UPnP with Python from <a href="https://www.electricmonk.nl/log/2016/07/05/exploring-upnp-with-python/">Ferry Boender blog</a>

<p>UPnP uses a variety of different protocols to accomplish its goals:</p>
<ul>
<li><b>SSDP</b>: <em>Simple Service Discovery Protocol</em>, for discovering UPnP devices on the local network.</li>
<li><b>SCPD</b>: <em>Service Control Point Definition</em>, for defining the actions offered by the various services.</li>
<li><b>SOAP</b>: <em>Simple&nbsp;Object Access Protocol</em>, for actually calling actions.</li>
</ul>
<img alt="" src="https://www.electricmonk.nl/log/wp-content/uploads/2012/10/upnp_overview.png" title="upnp_overview">

# Discovery : SSDP

In [30]:
import ssdp

In [31]:
devices = ssdp.discover("upnp:rootdevice")
assetUPnP = devices[8]

In [32]:
assetUPnP

<SSDPResponse(http://192.168.0.1:49000/igddesc.xml, upnp:rootdevice, uuid:75802409-bccb-40e7-8e6c-E0286D0372B3::upnp:rootdevice)>

In [33]:
assetUPnP.location

'http://192.168.0.1:49000/igddesc.xml'

# Description : SCPD

## SCPD, Phase I: Fetching and parsing the root SCPD file

In [34]:
from urllib.request import urlopen
from urllib.parse import urlparse
from xml.dom import minidom

def XMLGetNodeText(node):
    """
    Return text contents of an XML node.
    """
    text = []
    for childNode in node.childNodes:
        if childNode.nodeType == node.TEXT_NODE:
            text.append(childNode.data)
    return(''.join(text))

location = assetUPnP.location

# Fetch SCPD
response = urlopen(location)
root_xml = minidom.parseString(response.read())
response.close()

# Construct BaseURL
base_url_elem = root_xml.getElementsByTagName('URLBase')
if base_url_elem:
    base_url = XMLGetNodeText(base_url_elem[0]).rstrip('/')
else:
    url = urlparse(location)
    base_url = '%s://%s' % (url.scheme, url.netloc)

# Output Service info
for node in root_xml.getElementsByTagName('service'):
    service_type = XMLGetNodeText(node.getElementsByTagName('serviceType')[0])
    control_url = '%s%s' % (
        base_url,
        XMLGetNodeText(node.getElementsByTagName('controlURL')[0])
    )
    scpd_url = '%s%s' % (
        base_url,
        XMLGetNodeText(node.getElementsByTagName('SCPDURL')[0])
    )
    print('%s:\n  SCPD_URL: %s\n  CTRL_URL: %s\n' % (service_type,
                                                     scpd_url,
                                                     control_url))

urn:schemas-any-com:service:Any:1:
  SCPD_URL: http://192.168.0.1:49000/any.xml
  CTRL_URL: http://192.168.0.1:49000/igdupnp/control/any

urn:schemas-upnp-org:service:WANCommonInterfaceConfig:1:
  SCPD_URL: http://192.168.0.1:49000/igdicfgSCPD.xml
  CTRL_URL: http://192.168.0.1:49000/igdupnp/control/WANCommonIFC1

urn:schemas-upnp-org:service:WANDSLLinkConfig:1:
  SCPD_URL: http://192.168.0.1:49000/igddslSCPD.xml
  CTRL_URL: http://192.168.0.1:49000/igdupnp/control/WANDSLLinkC1

urn:schemas-upnp-org:service:WANIPConnection:1:
  SCPD_URL: http://192.168.0.1:49000/igdconnSCPD.xml
  CTRL_URL: http://192.168.0.1:49000/igdupnp/control/WANIPConn1

urn:schemas-upnp-org:service:WANIPv6FirewallControl:1:
  SCPD_URL: http://192.168.0.1:49000/igd2ipv6fwcSCPD.xml
  CTRL_URL: http://192.168.0.1:49000/igd2upnp/control/WANIPv6Firewall1



## SCPD, Phase II: Service SCPD files